# Speech Recognition Between YES and NO
Using the Librosa and the Conv2D package in Keras, make a prediction on whether someone has said the word 'yes' or the word 'no'

First, I loaded Keras and the necessary Keras packages:

In [1]:
import keras

Using TensorFlow backend.


In [2]:
import librosa
import librosa.display
import librosa.feature

In [3]:
import numpy as np 
import os
import random

## Obtaining the Dataset

The dataset was obtained from the Google Research Blog. Here is the link to the blog: https://research.googleblog.com/2017/08/launching-speech-commands-dataset.html

The dataset was downloaded as a .tar file so the file had to be untarred for the correct data to be extracted. The file contained many other words but only the 'yes' and 'no' folders were used for this project. The 'yes' and 'no' folders were copied into my local directory. The path to each audio file inside each folder were loaded into an array and that array was concatenated into one large array.

A model for speech recognition was found in this blog: 
https://blog.manash.me/building-a-dead-simple-word-recognition-engine-using-convnet-in-keras-25e72c19c12b
Modifications were made to the code as shown below:

In [4]:
directory_in_str = ("C:\\Users\\Trisha Kadle\\Documents\\Machine_Learning\\New folder\\yes")
fn = []
directory_in_str1 = ("C:\\Users\\Trisha Kadle\\Documents\\Machine_Learning\\New folder\\no")
fn1 = []

fn2 = []

directory = os.fsencode(directory_in_str) 
for file in os.listdir(directory): 
    filename = os.fsdecode(file)
    if filename.endswith(".wav"): 
        fn.append(os.path.join(directory_in_str, filename))
        
directory1 = os.fsencode(directory_in_str1) 
for file in os.listdir(directory1): 
    filename1 = os.fsdecode(file)
    if filename1.endswith(".wav"): 
        fn1.append(os.path.join(directory_in_str1, filename1))

fn2 = np.concatenate((fn, fn1),axis = 0)

## Getting the MFCC 

For each audio the MFCC (Mel-freqeuncy cepstral coefficients) had to be found. The MFCC are coefficients that easily map audio based on the different frequencies present in the audio. The following steps were taken to find the MFCC:
* Each audio file in fn2 was loaded using Librosa
* A downsample operation was performed to reduce the amount of computing power needed
* The MFCC was found using Librosa
* The MFCC vector was zero padded to so that respective MFCC vectors would be the same length
* The MFCC for that sample is saved in a larger MFCC_vec containing the MFCC values for all the audio samples


In [5]:
mfcc_vec = np.empty((len(fn2),20,11))

for file in range(len(fn2)):
    y, sr = librosa.load(fn2[file], mono=True, sr=None)
    y = y[::3]
    S = librosa.feature.mfcc(y, sr=16000)
    pad = 11-S.shape[1]
    S = np.pad(S,pad_width=((0,0),(0,pad)), mode='constant')
    mfcc_vec[file] = np.r_[S]
                  
np.save('yes_no.npy', mfcc_vec)


## Create a 'Labels' Array

An array containing the labels for each audio sample was created. If the audio sample was a 'yes', the corresponding label would be a 1. If the audio sample was a 'no', the corresponding label would be a 0. The number of yes and no labels to add were based on the length of the arrays containing to paths to each 'yes' and 'no' audio files. 

In [6]:
yes_and_no_labels = []
yes_labels = np.full(len(fn), 1)
no_labels = np.full(len(fn1), 0)
yes_and_no_labels = np.concatenate((yes_labels,no_labels),axis = 0)

## Create Training and Test Data

To sklearn package was used to create the training and test data. The built in function train_test_split was used to appropariately shuffle and then split the total data into 60% training data and 40% test data.
(The default value of shuffle is TRUE) 

In [7]:
import sklearn.model_selection
from sklearn.model_selection import train_test_split
X_tr, X_ts, y_tr, y_ts = train_test_split(mfcc_vec, yes_and_no_labels, test_size=0.4)


## Reshaping the data

In order to train the data using Conv2D, the data had to be reshaped to be four dimensional. To do this, the reshape function was used to add a fourth dimension to the already existing shape of (number_of_samples, 20, 11). 

In [8]:
from keras.utils import to_categorical

X_tr = X_tr.reshape(X_tr.shape[0], 20, 11, 1)
X_ts = X_ts.reshape(X_ts.shape[0], 20, 11, 1)


## Training the Data

The appropriate functions were loaded from Keras. 
Conv2D was used with two dense layers. The optimizer chosen was the Adam opitmizer and binary crossentropy was used because the desired results is either a 0 or 1. If more than two words were being classified, categorical crossentropy would have been used. The model summary is printed below. 

In [9]:
from keras.models import Model, Sequential
from keras.layers import Conv2D, Dropout, Flatten, Dense, Activation
from keras import optimizers

In [10]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(20, 11, 1)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))

opt = optimizers.Adam(lr=0.001) # beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 19, 10, 32)        160       
_________________________________________________________________
dropout_1 (Dropout)          (None, 19, 10, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6080)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               778368    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 778,657
Trainable params: 778,657
Non-trainable params: 0
_________________________________________________________________


## Getting the Model Accruacy 

The training data was then used to test the remaining data using the model created above. 100 Epochs were used with a batch size of 100. The model begins to saturate around Epoch 85 and ends with a final validation accuracy of 96.11%.

In [11]:
model.fit(X_tr, y_tr, batch_size=100, epochs=100, verbose=1, validation_data=(X_ts, y_ts))


Train on 2851 samples, validate on 1901 samples
Epoch 1/100
2851/2851 [==============================] - 3s 1ms/step - loss: 0.5951 - acc: 0.6998 - val_loss: 0.3857 - val_acc: 0.8611
Epoch 2/100
2851/2851 [==============================] - 2s 825us/step - loss: 0.3672 - acc: 0.8558 - val_loss: 0.2781 - val_acc: 0.8964
Epoch 3/100
2851/2851 [==============================] - 2s 816us/step - loss: 0.2836 - acc: 0.8888 - val_loss: 0.2347 - val_acc: 0.9095
Epoch 4/100
2851/2851 [==============================] - 2s 871us/step - loss: 0.2394 - acc: 0.9074 - val_loss: 0.2191 - val_acc: 0.9148
Epoch 5/100
2851/2851 [==============================] - 2s 818us/step - loss: 0.2205 - acc: 0.9158 - val_loss: 0.2002 - val_acc: 0.9221
Epoch 6/100
2851/2851 [==============================] - 2s 803us/step - loss: 0.1976 - acc: 0.9263 - val_loss: 0.1894 - val_acc: 0.9290
Epoch 7/100
2851/2851 [==============================] - 2s 800us/step - loss: 0.1773 - acc: 0.9323 - val_loss: 0.1898 - val_acc: 0.

2851/2851 [==============================] - 2s 801us/step - loss: 0.0096 - acc: 0.9986 - val_loss: 0.1119 - val_acc: 0.9642
Epoch 61/100
2851/2851 [==============================] - 2s 823us/step - loss: 0.0090 - acc: 0.9989 - val_loss: 0.1104 - val_acc: 0.9627
Epoch 62/100
2851/2851 [==============================] - 2s 843us/step - loss: 0.0098 - acc: 0.9986 - val_loss: 0.1124 - val_acc: 0.9621
Epoch 63/100
2851/2851 [==============================] - 2s 842us/step - loss: 0.0108 - acc: 0.9975 - val_loss: 0.1152 - val_acc: 0.9579
Epoch 64/100
2851/2851 [==============================] - 2s 813us/step - loss: 0.0096 - acc: 0.9986 - val_loss: 0.1178 - val_acc: 0.9590
Epoch 65/100
2851/2851 [==============================] - 2s 819us/step - loss: 0.0099 - acc: 0.9979 - val_loss: 0.1101 - val_acc: 0.9611
Epoch 66/100
2851/2851 [==============================] - 2s 850us/step - loss: 0.0088 - acc: 0.9993 - val_loss: 0.1143 - val_acc: 0.9632
Epoch 67/100
2851/2851 [=======================

## Test the Model With a New Audio File 

Using Pyaudio, a new 1 second audio file with a sampling rate of 16000 was recorded and saved. The recorded audio file can be played below. Then the MFCC of the new audio file is found using Librosa. This is reshaped and the model created above is used to predict whether the audio file is of a person saying 'yes' or saying 'no. The predicted value to rounded to either a 1 or a 0 and the respective word is printed.  

In [12]:
from IPython.display import Audio
import pyaudio
import wave

duration = 1  # seconds
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
DURATION = 1
BLOCKSIZE = 1024
WAVE_OUTPUT_FILENAME = "audio_samp.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=BLOCKSIZE)

print("Start Recording")

frames = []

for i in range(0, int(RATE / BLOCKSIZE * DURATION)):
    data = stream.read(BLOCKSIZE)
    frames.append(data)

print("Done")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

fn_samp = "C:\\Anaconda3\\Scripts\\audio_samp.wav"
Audio(fn_samp, rate=RATE)

Start Recording
Done


In [13]:
fn_samp = "C:\\Anaconda3\\Scripts\\audio_samp.wav"
mfcc_vec = np.empty((1,20,11))

y, sr = librosa.load(fn_samp, mono=True, sr=None)
y = y[::3]
S = librosa.feature.mfcc(y, sr=16000)
pad = 11-S.shape[1]
S = np.pad(S,pad_width=((0,0),(0,pad)), mode='constant')
mfcc_samp = S

sample_reshaped = mfcc_samp.reshape(1, 20, 11, 1)

output = np.around(model.predict(sample_reshaped))

if (output == 1):
    print("You said YES")
if (output == 0):
    print("You said NO")

You said NO
